In [ ]:
import sys
import time
import threading
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 模型版本随训练更新
model_path = "../../autodl-tmp/DiabetesPDiagLLM"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code=True)
tokenizer.padding_side="left"
tokenizer.pad_token="[PAD]"
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()

def loading_indicator():
    """Displays a loading indicator while waiting for the LLM response."""
    while not stop_loading:
        sys.stdout.write("\rThinking... ")
        sys.stdout.flush()
        time.sleep(0.5)
    sys.stdout.write("\r")
    sys.stdout.flush()

def query_llm(prompt):
    global stop_loading
    stop_loading = False
    loader_thread = threading.Thread(target=loading_indicator)
    loader_thread.start()
    """Sends the prompt to the LLM and returns the response."""
    messages = [{
                "role": "system",
                "content": "你是一个专业的医生，你的任务是：1.风险评估：根据患者病情病例，输出相应的医疗风险评估。2.健康建议：结合患者的年龄、近期血糖控制情况、血压等并发症情况、用药与依从性情况等，给出专业的健康建议。3.饮食运动建议：根据患者 BMI 以及体检情况计算每日建议摄入的卡路里，结合患者体质给出每日运动相关建议。请基于诊疗指南，为以下患者提供综合的管理意见（不要输出重复内容）:"
            },
            {
                "role": "user",
                "content": prompt
            }
           ]

    inputs = tokenizer.apply_chat_template(messages,
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )
    inputs = inputs.to(device)

    gen_kwargs = {"max_length": 1024, "do_sample": True, "top_k": 1}
    # 在输出长度不够的时候更改max_length
    output = ""
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        output =  tokenizer.decode(outputs[0], skip_special_tokens=True)

    stop_loading = True
    loader_thread.join()
    return output

while True:
    user_input = input("\nEnter your prompt (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        print("Have a nice day!")
        break
    response = query_llm(user_input)
    print("\nLLM Response:", response)
    print("*内容由AI生成,仅供参考,请结合医生诊断采用,祝您身体健康*\n")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


Enter your prompt (or type 'exit' to quit):  基本情况：年龄：65；性别：男；类型：红标。监测情况：第1次回访：监测内容：血糖；空腹空腹血糖为9.0mmol/L；第2次回访：监测内容：体征（身高、体重），血压；收缩压为111.0mmHg；舒张压为66.0mmHg；BMI为20.42；第3次回访：监测内容：血糖；早餐后非空腹血糖为8.7mmol/L。


Thinking... 
LLM Response: 
您的第1次反馈结果如下：
医生针对您的情况进行的风险评估：10年心脑血管意外风险（ICVD改良，例如心梗和卒中等事件风险）评估为0%～0%，是同龄健康人的0倍，但积极的综合干预能使风险减少0%左右，降低到0%～0%。
您当前的血糖异常情况为：正常
建议监测时间为：早餐后、空腹
对您提出的健康建议如下：1.结合您的年龄，近期血糖控制尚可，建议继续监测空腹血糖和餐后2小时血糖，近期可检测糖化血红蛋白，了解近3个月的血糖水平，并随时联系专科医师复诊；
2.坚持规律用药，注意测量血压；
3.注意糖尿病饮食，适量运动（参考随后为您出具的糖尿病饮食方案和运动方案）； 
4.保持心情舒畅，定期复查，如有不适及时就医。
建议您每日总卡路里摄入在2250左右，
每日碳水化合物摄入281至366克，
每日蛋白质摄入84至113克，
每日脂肪摄入50至75克，
建议您每日通过运动消耗147.0卡路里。建议您每日通过运动消耗147.0卡路里。对您提出的运动建议如下：如何选择适合自己的运动：
在选择运动方式时，要考虑到年龄、性别、体质、生活方式等的不同，因人而异、因时制宜，选择个体化的运动方式。
总体来说，糖尿病患者的运动应具备三个特点: 适量、全身性、有节奏。有氧运动就同时具备这三个特点，而且强度低、持续时间长，可以达到让人呼吸急促又不会气喘吁吁的程度，使人微微出汗又不会大汗淋漓。
综合来说，适合大多数糖尿病患者的运动有做操、打拳、慢跑、快走、踢建子、打羽毛球、打乒乓球、跳交谊舞等，既能锻炼身体，又能让人放松心情，还便于长期坚持。运动时的心率一般控制在每分钟130到150下，年龄越大，心率控制得越低。
运动前自测血压，如高于140/90毫米汞柱，不适宜进行运动；如低于130/80毫米汞柱，可适当增加运动强度。运动后自测血压，如高于180/100毫米汞柱，应立即停止运动，待血压恢复正常后再继续运动。
您当前的血糖异常情况为：正常
建议监测时间为：早餐后、空腹
对您提出的健康建议如下：1.结合您的年龄，近期血糖控制尚可，建议继续监测空腹血糖和餐后2小时血糖，近期可检测糖化血红蛋白，了解近3个月的血糖水平，并随时联系专科医师复诊；
2.坚持规律用药，注意测量血压；
3.注意糖尿病饮食，适量运动（参考随后为您出具的糖尿病饮食方案和运动方案）；


Enter your prompt (or type 'exit' to quit):  基本情况：年龄：65；性别：男；类型：红标。监测情况：第1次回访：监测内容：血糖；空腹空腹血糖为9.0mmol/L；第2次回访：监测内容：体征（身高、体重），血压；收缩压为111.0mmHg；舒张压为66.0mmHg；BMI为20.42；第3次回访：监测内容：血糖；早餐后非空腹血糖为8.7mmol/L。


Thinking... 